# Clone, Install Import

## Timestamp this run

In [1]:
!pip install pytz

In [2]:
# reference: https://www.programiz.com/python-programming/datetime/current-time

from datetime import datetime
import pytz

print(datetime.now(pytz.timezone('America/New_York')).strftime('%m/%d(%a)_%H:%M:%S'))

11/19(Sat)_09:32:54


# Wandb install, login, import

In [3]:
!pip install wandb

In [4]:
!wandb login "6f19b1e6735ebc69af24f18d5b426262416027fb"

wandb: Appending key for api.wandb.ai to your netrc file: /home/studio-lab-user/.netrc


In [5]:
import wandb

## Torchsummary Install, Import

In [6]:
!pip install torch-summary==1.4.5

In [7]:
from torchsummary import summary

summary

<function torchsummary.torchsummary.summary(model: torch.nn.modules.module.Module, input_data: Union[torch.Tensor, torch.Size, Sequence[torch.Tensor], Sequence[Union[int, Sequence[Any], torch.Size]], NoneType] = None, *args: Any, batch_dim: Optional[int] = 0, branching: bool = True, col_names: Optional[Iterable[str]] = None, col_width: int = 25, depth: int = 3, device: Optional[torch.device] = None, dtypes: Optional[List[torch.dtype]] = None, verbose: int = 1, **kwargs: Any) -> torchsummary.model_statistics.ModelStatistics>

## Clone team's code

In [8]:
!rm -r /content/csgy6953_DeepLearning_Midterm/

rm: cannot remove '/content/csgy6953_DeepLearning_Midterm/': No such file or directory


In [9]:
!git clone -b config_kernel_size_on_updated_main "https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"

fatal: destination path 'csgy6953_DeepLearning_Midterm' already exists and is not an empty directory.


if running on Google Colab:

In [10]:
# !cp -r /content/csgy6953_DeepLearning_Midterm/src/ .

if running on Amazon SageMaker Studio Lab:

In [11]:
!cp -r csgy6953_DeepLearning_Midterm/src/ .

# Import, Seed, Device

In [12]:
import torch
import torch.nn as nn

import time
import random

In [13]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Load Data

In [15]:
from src.data import get_transformed_data, make_data_loaders
from src.transforms import make_auto_transforms # used to use: make_transforms

BATCH_SIZE = 128
VALID_RATIO = 0.1

train_data, valid_data, test_data = \
get_transformed_data(make_transforms=make_auto_transforms, valid_ratio=VALID_RATIO)

train_iter, valid_iter, test_iter = \
make_data_loaders(train_data, valid_data, test_data, BATCH_SIZE)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Model

## Architecture <<<

In [17]:
from src.model import StemConfig, ResidualBlockType, ResNet

DROPOUT = 0.1
MAIN_BLOCK_KERNEL_SIZE = 2

# 1 tuple == 1 layer
# block with or without bottleneck, how many blocks in each layer, out_channels, dropout prob, kernel_size
architecture = [
    (ResidualBlockType.BASIC, 2,  64, DROPOUT, MAIN_BLOCK_KERNEL_SIZE),
    (ResidualBlockType.BASIC, 2, 128, DROPOUT, MAIN_BLOCK_KERNEL_SIZE),
    (ResidualBlockType.BASIC, 2, 256, DROPOUT, MAIN_BLOCK_KERNEL_SIZE),
    (ResidualBlockType.BASIC, 2, 512, DROPOUT, MAIN_BLOCK_KERNEL_SIZE),
]

# 2022.11/18(5)_p03.59 (Oscar)
# slightly reduce the last layer's out_channels to keep overall params under 5M:
# 512 -> 5,069,130
# 508 -> 5,014,978
# 507 -> 5,001,500
# 506 -> 4,988,046 <<<

In [18]:
config = StemConfig(num_channels=64, kernel_size=3, stride=1, padding=1)

model  = ResNet(architecture, stem_config=config, output_size=10)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


## Initialize model shapes

In [19]:
# intialize a new model

inputs = torch.empty((BATCH_SIZE, 3, 32, 32)) #  passed
inputs.normal_()

model = model.to(device)

outputs = model(inputs.to(device)) # internally converts all nn.LazyConv2d layers to nn.Conv2d

print('-------------------')
print('-------------------')

print(model)

from src.utils import count_parameters

print(count_parameters(model))
print(outputs.size())

-------------------
-------------------
ResNet(
  (stem): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (classifier): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
  (body): Sequential(
    (block_2): Sequential(
      (0): ResidualBlock(
        (dropout): Dropout(p=0.1, inplace=False)
        (conv1): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
        (conv2): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1), bias=False)
        (relu): ReLU(inplace=True)
        (out): ReLU(inplace=True)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

## Count parameters

In [20]:
num_parameters, num_parameters_requiring_grad = count_parameters(model)
print(num_parameters)

5069130


In [21]:
QUIET_VERBOSE = 0

one_sample_input_shape = (3, 32, 32)

summary(model, one_sample_input_shape, verbose = QUIET_VERBOSE)

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 32, 32]          --
|    └─Conv2d: 2-1                       [-1, 64, 32, 32]          1,728
|    └─BatchNorm2d: 2-2                  [-1, 64, 32, 32]          128
|    └─ReLU: 2-3                         [-1, 64, 32, 32]          --
├─Sequential: 1-2                        [-1, 512, 4, 4]           --
|    └─Sequential: 2-4                   [-1, 64, 32, 32]          --
|    |    └─ResidualBlock: 3-1           [-1, 64, 32, 32]          33,024
|    |    └─ResidualBlock: 3-2           [-1, 64, 32, 32]          33,024
|    └─Sequential: 2-5                   [-1, 128, 16, 16]         --
|    |    └─ResidualBlock: 3-3           [-1, 128, 16, 16]         107,008
|    |    └─ResidualBlock: 3-4           [-1, 128, 16, 16]         131,584
|    └─Sequential: 2-6                   [-1, 256, 8, 8]           --
|    |    └─ResidualBlock: 3-5           [-1, 256, 8, 8]       

## Initialize parameters

In [22]:
from src.utils import initialize_parameters

# initialize model weights
model.apply(initialize_parameters);

# Train, validate, log

## Loss function

In [23]:
criterion = nn.CrossEntropyLoss().to(device)

## Optimizer, scheduler (learning rate)

In [24]:
from torch.optim.lr_scheduler import CyclicLR

learning_rate = 0.01
max_lr        = 0.5
base_lr       = 0.0001

lr_schedule   = 'CyclicLR_base_0.0001_max_0.5'

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=learning_rate, cycle_momentum=False)

## Iterate!

In [25]:
from src.engine import train_one_epoch, evaluate

In [31]:
EPOCHS  = 200

In [32]:
## setup wandb logging

assert 'wandb' in locals() # 'wandb' has been imported already

ENTITY_NAME  = 'dlf22_mini_project' # this is our team name

# PROJECT_NAME = 'Junk_Test_Project'
PROJECT_NAME = 'ResNet_5M'

# RUN_NAME     = 'resnet_osca_kernel_size_2_2022.11.18.p04.13'
RUN_NAME = 'osca_kernel_size_2_replicate_resnet18_adamw'

WANDB_CONFIG = \
{"architecture"          : architecture,                 # the model
 "num_params"            : num_parameters,                # the model
 "grad_params"           : num_parameters_requiring_grad, # the model
 "main_block_kernel_size": MAIN_BLOCK_KERNEL_SIZE,        # the model
 "dropout"       : DROPOUT,         # the model
 "learning_rate" : learning_rate,   # how to gradient descent
 "lr_schedule"   : lr_schedule,     # how to gradient descent
 "batch_size"    : BATCH_SIZE,      # how to gradient descent
 "epochs"        : EPOCHS,}         # train for how long

WANDB_CONFIG

{'architecture': [(<ResidualBlockType.BASIC: 0>, 2, 64, 0.1, 2),
  (<ResidualBlockType.BASIC: 0>, 2, 128, 0.1, 2),
  (<ResidualBlockType.BASIC: 0>, 2, 256, 0.1, 2),
  (<ResidualBlockType.BASIC: 0>, 2, 512, 0.1, 2)],
 'num_params': 5069130,
 'main_block_kernel_size': 2,
 'dropout': 0.1,
 'learning_rate': 0.01,
 'batch_size': 128,
 'epochs': 3}

In [33]:
wandb\
.init(name=RUN_NAME, project=PROJECT_NAME, entity=ENTITY_NAME,config=WANDB_CONFIG)

In [34]:
# save to disk the "best" model === the model with the lowest validation loss
best_model_path = RUN_NAME + '.pt'
print(best_model_path)

osca_kernel_size_2_replicate_resnet18_adamw.pt


In [35]:
from src.utils import epoch_time

FIRST_EPOCH = 1

best_loss = float('inf')
for epoch in range(FIRST_EPOCH, FIRST_EPOCH + EPOCHS):

    print(f"Epoch {epoch}")
    
    ## TRAIN
    start = time.time()
    # train
    train_loss, train_acc  = train_one_epoch(model, train_iter, criterion, optimizer, device)
    # log & echo
    train_mins, train_secs = epoch_time(start, time.time())
    wandb.log({"train_loss": train_loss,
               "train_acc" : train_acc,
               "epoch"     : epoch})
    print(f"\tTrain elapsed: {train_mins}:{train_secs}, loss: {train_loss:.4f}, acc: {train_acc * 100:.2f}%")

    ## VALIDATE
    start = time.time()
    # validate
    val_loss, val_acc  = evaluate(model, valid_iter, criterion, device)
    # log & echo   
    val_mins, val_secs = epoch_time(start, time.time())
    wandb.log({"val_loss": val_loss,
               "val_acc" : val_acc,
               "epoch"   : epoch,})
    print(f"\tValidation elapsed: {val_mins}:{val_secs}, loss: {val_loss:.4f}, acc: {val_acc * 100:.2f}%")
    
    scheduler.step()
    
    current_lr = scheduler.get_last_lr()[0]
    wandb.log({"current_lr": current_lr,
               "current_learning_rate": current_lr,
               "epoch"     : epoch})
    
    ## Memorize "best" model === the model with the lowest validation loss
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), best_model_path)

Epoch 1
	Train elapsed: 1:5, loss: 2.0357, acc: 27.80%
	Validation elapsed: 0:1, loss: 1.6630, acc: 39.04%
0.00010495000000000166
Epoch 2
	Train elapsed: 1:2, loss: 1.6995, acc: 38.63%
	Validation elapsed: 0:1, loss: 1.4054, acc: 49.10%
0.00010989999999999892
Epoch 3
	Train elapsed: 1:4, loss: 1.5574, acc: 43.62%
	Validation elapsed: 0:1, loss: 1.2502, acc: 54.84%
0.00011485000000000057


# Test, log

In [36]:
print(best_model_path),
model.load_state_dict(torch.load(best_model_path))

osca_kernel_size_2_replicate_resnet18_adamw.pt


<All keys matched successfully>

In [37]:
test_loss, test_acc = evaluate(model.to(device), test_iter, criterion, device)
print(f"Test Loss: {test_loss:.4f}\nTest Accuracy: {test_acc * 100:.2f}%")

wandb.log({
    "test_loss": test_loss,
    "test_acc" : test_acc,
})

Test Loss: 1.2691
Test Accuracy: 53.74%
